#### Creating Lable & Label Junction Dataframes

In [0]:
# Find entries with label from the two objects that may contain them
df = spark.sql("SELECT * FROM data").filter(
    (size(col("payload_issue.labels")) > 0)
    | (size(col("payload_pull_request.labels")) > 0)
)

In [0]:
cols = ["id", "color", "default", "description", "name", "node_id", "url"]

In [0]:
issue_labels = df.select(explode(col("payload_issue.labels"))).select("col.*")
pull_request_labels = df.select(explode(col("payload_pull_request.labels"))).select("col.*")

#### Label

In [0]:
labelDF = issue_labels.union(pull_request_labels).select(cols).distinct().orderBy("id")

#### Label junction tables

In [0]:
issue_labelDF = (
    df.select("payload_issue.id", explode(col("payload_issue.labels")))
    .select(col("id").alias("issue_id"), col("col.id").alias("label_id"))
    .distinct()
)
pull_request_labelDF = (
    df.select("payload_pull_request.id", explode(col("payload_pull_request.labels")))
    .select(col("id").alias("pull_request_id"), col("col.id").alias("label_id"))
    .distinct()
)

#### Write to database

In [0]:
labelDF.repartition(2).write.mode("overwrite").parquet("abfss://team1-project2@20230821desa.dfs.core.windows.net/SilverLayer/label")

In [0]:
issue_labelDF.repartition(1).write.mode("overwrite").parquet("abfss://team1-project2@20230821desa.dfs.core.windows.net/SilverLayer/issue-label")

In [0]:
pull_request_labelDF.repartition(1).write.mode("overwrite").parquet("abfss://team1-project2@20230821desa.dfs.core.windows.net/SilverLayer/pull-request-label")